In [4]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using LinearAlgebra, Plots
using ProgressMeter
import ForwardDiff as FD
import Convex as cvx
import ECOS

include(joinpath(@__DIR__,"utils","fmincon.jl"))

  Activating environment at `c:\CMU_Spring\OptimalSailboat\Project.toml`


fmincon (generic function with 1 method)